In [6]:
import json
import os
from pathlib import Path
from slugify import slugify
import boto3
from datetime import datetime, timezone
import tempfile

In [7]:
def create_folder(path: str | None) -> str | None:
    if not path:
        return None

    Path(path).mkdir(parents=True, exist_ok=True)

    return path

In [8]:
def create_json(name: str, description: str, source: str, films: list) -> str:
    return json.dumps({
            "name": name,
            "description": description,
            "source": source,
            "updated": datetime.now(timezone.utc).isoformat(),
            "films": films
        }, indent=True)

In [9]:
def save_to_s3(file: Path, filename: str | None):
    s3 = boto3.resource('s3',
        endpoint_url = os.getenv("R2_ENDPOINT"),
        aws_access_key_id = os.getenv("R2_ACCESS_KEY_ID"),
        aws_secret_access_key = os.getenv("R2_ACCESS_KEY_SECRET")
    )

    if filename is None:
        filename = file.name
 
    s3.Object(os.getenv("R2_BUCKET_NAME"), filename).upload_file(str(file)) # type: ignore // boto3 appears to be poorly typed

In [10]:
def save(name: str, description: str, source: str, films: list):
    temp_file = tempfile.NamedTemporaryFile()

    with open(temp_file.name, "w") as file:
        file.write(create_json(name, description, source, films))
    
    save_to_s3(Path(temp_file.name), slugify(name) + ".json")

    # output R2 URL
    public_endpoint = os.getenv("R2_PUBLIC_URL")
    if public_endpoint:
        print(public_endpoint + "/" + slugify(name) + ".json")